In [ ]:
#JJ

## Imports

In [ ]:
# for colab run
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/projeto/automl-dynamic-search-space/analysis/dynamic-search-space/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

COLORS = ["#000057", "#4d759a", "#3282B8",  "#999ea2", "#4f4f4f", "#1B262C"]

MY_PALLETE = sns.color_palette("colorblind")
sns.set_palette(MY_PALLETE)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.tree import DecisionTreeRegressor
import pickle

In [ ]:
def pipeline_statistics_meta_features(meta_knowledge, exclusion_list=None):
  """ Use exclusion_list with list of dataset to remove."""

  if exclusion_list:
    idx = ~meta_knowledge["dataset"].isin(exclusion_list)
    meta_knowledge = meta_knowledge[idx]

  aux_cl = meta_knowledge \
      .groupby(["classifier"]) \
      .agg(
          cl_min_perf=("median_perf", "min"),
          cl_max_perf=("median_perf", "max"),
          cl_median_perf=("median_perf", "median"),
          cl_mean_perf=("median_perf", "mean"),
          cl_std_perf=("median_perf", "std")
      )

  aux_pr =  meta_knowledge \
      .groupby(["preprocessor"]) \
      .agg(
          pr_min_perf=("median_perf", "min"),
          pr_max_perf=("median_perf", "max"),
          pr_median_perf=("median_perf", "median"),
          pr_mean_perf=("median_perf", "mean"),
          pr_std_perf=("median_perf", "std")
      )

  aux_p =  meta_knowledge \
      .groupby(["classifier", "preprocessor"]) \
      .agg(
          p_min_perf=("median_perf", "min"),
          p_max_perf=("median_perf", "max"),
          p_median_perf=("median_perf", "median"),
          p_mean_perf=("median_perf", "mean"),
          p_std_perf=("median_perf", "std")
      )

  return aux_cl, aux_pr, aux_p

def generate_meta_dataset(D_train, d_test, meta_knowledge, meta_target, meta_feature):
  minn = -1e10
  maxx = 1e10

  # fill pipeline without performance with 0
  # fill nan meta-feature with 0 and replace +-inf
  meta_dataset_ = meta_target.fillna(0)\
    .merge(
        meta_feature\
          .replace([np.inf], maxx)\
          .replace([-np.inf], minn),
        how="inner", on="dataset")\
    .fillna(0.0)

  aux_cl, aux_pr, aux_p = pipeline_statistics_meta_features(
        meta_knowledge, exclusion_list=d_test)

  # fill nan performance with 0
  meta_dataset_ = meta_dataset_\
    .merge(aux_cl, on="classifier", how="left")\
    .merge(aux_pr, on="preprocessor", how="left")\
    .merge(aux_p, on=["classifier", "preprocessor"], how="left")\
    .fillna(0.0)

  meta_dataset_["classifier-num"] = meta_dataset_["classifier"].astype("category").cat.codes
  meta_dataset_["preprocessor-num"] = meta_dataset_["preprocessor"].astype("category").cat.codes

  train_idx = meta_dataset_["dataset"].isin(D_train)
  test_idx = meta_dataset_["dataset"].isin(d_test)

  return meta_dataset_[train_idx], meta_dataset_[test_idx]

def load_meta_model(name):
  with open(name, 'rb') as f:
      return pickle.load(f)

In [ ]:
D = ['dataset_1039',
 'dataset_1044',
 'dataset_1046',
 'dataset_1049',
 'dataset_1050',
 'dataset_1053',
 'dataset_1056',
 'dataset_1063',
 'dataset_1068',
 'dataset_1069',
 'dataset_11',
 'dataset_1116',
 'dataset_1120',
 'dataset_1128',
 'dataset_1130',
 'dataset_1134',
 'dataset_1142',
 'dataset_1146',
 'dataset_1161',
 'dataset_1166',
 'dataset_1233',
 'dataset_1457',
 'dataset_1459',
 'dataset_1462',
 'dataset_1466',
 'dataset_1471',
 'dataset_1475',
 'dataset_1478',
 'dataset_1479',
 'dataset_1480',
 'dataset_1481',
 'dataset_1483',
 'dataset_1485',
 'dataset_1487',
 'dataset_1491',
 'dataset_1494',
 'dataset_1496',
 'dataset_1497',
 'dataset_15',
 'dataset_1501',
 'dataset_1502',
 'dataset_1503',
 'dataset_1507',
 'dataset_1509',
 'dataset_151',
 'dataset_1510',
 'dataset_1515',
 'dataset_1528',
 'dataset_1529',
 'dataset_1530',
 'dataset_1531',
 'dataset_1532',
 'dataset_1535',
 'dataset_1536',
 'dataset_1538',
 'dataset_1541',
 'dataset_1542',
 'dataset_1547',
 'dataset_1549',
 'dataset_155',
 'dataset_1552',
 'dataset_1553',
 'dataset_1590',
 'dataset_181',
 'dataset_182',
 'dataset_184',
 'dataset_185',
 'dataset_188',
 'dataset_2',
 'dataset_23',
 'dataset_23380',
 'dataset_23381',
 'dataset_24',
 'dataset_26',
 'dataset_279',
 'dataset_28',
 'dataset_30',
 'dataset_300',
 'dataset_307',
 'dataset_310',
 'dataset_311',
 'dataset_32',
 'dataset_333',
 'dataset_334',
 'dataset_335',
 'dataset_37',
 'dataset_375',
 'dataset_377',
 'dataset_40496',
 'dataset_40498',
 'dataset_40499',
 'dataset_40536',
 'dataset_40646',
 'dataset_40647',
 'dataset_40648',
 'dataset_40649',
 'dataset_40650',
 'dataset_40668',
 'dataset_40670',
 'dataset_40672',
 'dataset_40677',
 'dataset_40680',
 'dataset_40691',
 'dataset_40693',
 'dataset_40701',
 'dataset_40704',
 'dataset_40705',
 'dataset_40706',
 'dataset_40900',
 'dataset_40922',
 'dataset_40966',
 'dataset_40971',
 'dataset_40982',
 'dataset_40983',
 'dataset_40985',
 'dataset_40994',
 'dataset_41082',
 'dataset_41084',
 'dataset_41144',
 'dataset_41145',
 'dataset_41146',
 'dataset_41150',
 'dataset_41162',
 'dataset_41163',
 'dataset_4134',
 'dataset_41671',
 'dataset_41972',
 'dataset_41989',
 'dataset_41990',
 'dataset_42',
 'dataset_42193',
 'dataset_44',
 'dataset_451',
 'dataset_4534',
 'dataset_4538',
 'dataset_4541',
 'dataset_458',
 'dataset_46',
 'dataset_469',
 'dataset_470',
 'dataset_50',
 'dataset_57',
 'dataset_6',
 'dataset_60',
 'dataset_6332',
 'dataset_715',
 'dataset_717',
 'dataset_722',
 'dataset_725',
 'dataset_727',
 'dataset_728',
 'dataset_734',
 'dataset_735',
 'dataset_737',
 'dataset_740',
 'dataset_742',
 'dataset_750',
 'dataset_752',
 'dataset_757',
 'dataset_761',
 'dataset_770',
 'dataset_772',
 'dataset_799',
 'dataset_802',
 'dataset_803',
 'dataset_807',
 'dataset_816',
 'dataset_819',
 'dataset_821',
 'dataset_823',
 'dataset_825',
 'dataset_826',
 'dataset_833',
 'dataset_837',
 'dataset_839',
 'dataset_841',
 'dataset_846',
 'dataset_847',
 'dataset_871',
 'dataset_881',
 'dataset_884',
 'dataset_886',
 'dataset_897',
 'dataset_901',
 'dataset_903',
 'dataset_920',
 'dataset_923',
 'dataset_930',
 'dataset_934',
 'dataset_936',
 'dataset_937',
 'dataset_940',
 'dataset_947',
 'dataset_949',
 'dataset_950',
 'dataset_951',
 'dataset_981']

META_FEATURES_DEF = [
'nodes_per_inst',
 'joint_ent.mean',
 'nr_cat',
 'class_ent',
 'attr_ent.mean',
 'nr_num',
 'w_lambda',
 'var_importance.sd',
 'attr_to_inst',
 'nr_attr',
 'freq_class.sd'
]

META_FEATURES_PIP = [
 'pr_median_perf',
 'pr_mean_perf',
 'pr_std_perf',
 'cl_mean_perf',
 'cl_median_perf',
 'cl_std_perf',
 'p_mean_perf',
 'p_max_perf',
 'p_median_perf',
 'p_min_perf',
 'p_std_perf',
 'pr_min_perf',
 'preprocessor-num',
 'classifier-num',
]


META_FEATURES = META_FEATURES_PIP + META_FEATURES_DEF

In [ ]:
SEED=10
SEEDS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] # for repetition
D_train, d_test = train_test_split(D, test_size=0.20, shuffle=True, random_state=SEED)
CV = 10

In [ ]:
EXP_PATH = "experiments/"
if not os.path.exists(EXP_PATH):
  os.makedirs(EXP_PATH)
EXP_PIP_PATH = EXP_PATH + "pipeline_recomendation/"
if not os.path.exists(EXP_PIP_PATH):
  os.makedirs(EXP_PIP_PATH)

In [ ]:
MTF_TIME_SPENT = pd.read_csv("../meta-feature-analysis/meta_feature_time_spent.csv")
META_TARGET = pd.read_csv("../meta-model-design/meta-target-pipeline-f1-weighted.csv")
META_FEATURES_ALL = pd.read_csv("../meta-feature-analysis/meta_feature_dataset.csv")
meta_knowledge_ = pd.read_csv("../meta-model-design/meta_knowledge_cv_f1_weighted.csv")
D_TRAIN_META, D_TEST_META = generate_meta_dataset(D_train, d_test, meta_knowledge_, META_TARGET, META_FEATURES_ALL)
META_KNOWLEDGE = meta_knowledge_[["dataset", "preprocessor", "classifier","f1_weighted_train", "f1_weighted_val", "f1_weighted_test", "median_perf", "sum_cv_duration"]]\
.fillna(0.0)\
.rename(columns={"median_perf": "cv_median_f1_weighted"})
META_MODEL, META_MODEL_FEATURE = load_meta_model("../meta-model-design/meta_model_25_rem.pkl")

<ipython-input-8-4c0edf90d28e>:4: DtypeWarning: Columns (197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_knowledge_ = pd.read_csv("../meta-model-design/meta_knowledge_cv_f1_weighted.csv")


In [ ]:
META_KNOWLEDGE

,dataset,preprocessor,classifier,f1_weighted_train,f1_weighted_val,f1_weighted_test,cv_median_f1_weighted,sum_cv_duration
0,dataset_40985,select_percentile_classification,passive_aggressive,0.001842,0.001849,0.001835,0.001849,19.662576
1,dataset_40985,select_rates_classification,passive_aggressive,0.011496,0.011535,0.011177,0.011511,20.034161
2,dataset_40985,pca,decision_tree,0.004417,0.004338,0.003892,0.004391,13.993943
3,dataset_40985,feature_agglomeration,qda,0.000000,0.000000,0.000000,0.000000,10.278637
4,dataset_40985,kernel_pca,lda,0.000000,0.000000,0.000000,0.000000,228.333684
...,...,...,...,...,...,...,...,...
103702,dataset_1161,extra_trees_preproc_for_classification,decision_tree,0.954733,0.966579,0.916972,0.954440,82.555831
103703,dataset_1161,polynomial,k_nearest_neighbors,0.000000,0.000000,0.000000,0.000000,23.852781
103704,dataset_1161,select_percentile_classification,gradient_boosting,0.000000,0.000000,0.000000,0.000000,24.803508
103705,dataset_1161,kernel_pca,passive_aggressive,0.000000,0.000000,0.000000,0.000000,81.844108


In [ ]:
def compute_meta_features(dataset):
  query = f"dataset == '{dataset}'"
  mtf_ = D_TEST_META.query(query)
  time_ = MTF_TIME_SPENT.query(query)[META_FEATURES_DEF].fillna(0.0).sum(axis=1, numeric_only=True).values[0]
  return mtf_, time_


def dynamic_search_space_generation(omega, mtf):
  mtf_ = mtf.copy()
  y_pred = META_MODEL.predict(mtf_[META_MODEL_FEATURE])
  mtf_["predicted_performance"] = y_pred
  qt = mtf_["predicted_performance"].quantile([omega]).iloc[0]
  mask = mtf_["predicted_performance"] > qt
  mtf_ = mtf_[mask]
  preprocessor_ = mtf_["preprocessor"].unique().tolist()
  classifiers_ = mtf_["classifier"].unique().tolist()
  return preprocessor_, classifiers_

def compute_random_search(preprocessors, classifiers, dataset, mtf_time=0, seed=0):
  mask = (META_KNOWLEDGE["dataset"] == dataset) & (META_KNOWLEDGE["preprocessor"].isin(preprocessors)) & (META_KNOWLEDGE["classifier"].isin(classifiers))
  meta_knowledge_ = META_KNOWLEDGE[mask].sample(frac = 1, random_state=seed)
  meta_knowledge_["cumulative_sum_cv_duration"] = meta_knowledge_["sum_cv_duration"].cumsum()
  meta_knowledge_["cumulative_sum_cv_duration_mtf"] = meta_knowledge_["sum_cv_duration"].cumsum() + mtf_time
  return meta_knowledge_

def save_score_board(score_board, path):
  score_board.to_csv(path, index=False)

def autosklearn_2_meta():
  aux = D_TRAIN_META.copy()
  idx = aux.groupby(["dataset"])["f1_weighted_test"].idxmax()
  aux = aux.loc[idx]
  classifiers = aux["classifier"].unique().tolist()
  preprocessors = aux["preprocessor"].unique().tolist()

  return preprocessors, classifiers, 0.0


def landmarking_meta(dataset):
  METAFEATURES_LANDMARKING = [
    'best_node.mean',
    'best_node.sd',
    'elite_nn.mean',
    'elite_nn.sd',
    'linear_discr.mean',
    'linear_discr.sd',
    'naive_bayes.mean',
    'naive_bayes.sd',
    'one_nn.mean',
    'one_nn.sd',
    'random_node.mean',
    'random_node.sd',
    'worst_node.mean',
    'worst_node.sd',
  ]

  base_ = D_TRAIN_META.groupby(["dataset"]).first()[METAFEATURES_LANDMARKING]
  query_ =  D_TEST_META[D_TEST_META["dataset"] == dataset].head(1)[METAFEATURES_LANDMARKING]
  similar_dataset = (base_ - query_.iloc[0, :]).pow(2).sum(axis=1).pow(1/2).sort_values().index[0]

  aux = D_TRAIN_META.copy()
  idx = aux.groupby(["dataset"])["f1_weighted_test"].idxmax()

  aux = aux.loc[idx].query(f"dataset == '{similar_dataset}'")
  classifiers = aux["classifier"].unique().tolist()
  preprocessors = aux["preprocessor"].unique().tolist()

  time_ = MTF_TIME_SPENT.query(f"dataset == '{dataset}'")[METAFEATURES_LANDMARKING].fillna(0.0).sum(axis=1, numeric_only=True).values[0]

  return preprocessors, classifiers, time_

def random_meta(dataset, seed=0):
  classifiers = D_TRAIN_META["classifier"].sample(frac=1.0, random_state=seed).unique().tolist()[0]
  preprocessors = D_TRAIN_META["preprocessor"].sample(frac=1.0, random_state=seed).unique().tolist()[0]
  return [preprocessors], [classifiers], 0.0


def oracle_meta(dataset):
  idx = D_TEST_META.query(f"dataset == '{dataset}'").groupby(["dataset"])["f1_weighted_test"].idxmax()
  aux_ = D_TEST_META.loc[idx].head(1)
  classifiers = aux_["classifier"].unique().tolist()
  preprocessors = aux_["preprocessor"].unique().tolist()

  return preprocessors, classifiers, 0.0


In [ ]:
def meta_automl(dataset, omega, seed):
  mtf, mtf_time = compute_meta_features(dataset)
  preprocessors, classifiers = dynamic_search_space_generation(omega, mtf)
  return compute_random_search(preprocessors, classifiers, dataset, mtf_time, seed)


def run_meta_automl_experiment(omegas):
  for o in omegas:
    r = []
    for seed in SEEDS:
      aux = pd.concat([meta_automl(d, o, seed) for d in D_TEST_META["dataset"].unique()])
      aux["seed"] = seed
      r.append(aux)
    r = pd.concat(r)
    r.to_csv(EXP_PATH + f"meta_automl_omega_{o}.csv", index=False)


def rs_automl(dataset, seed):
  preprocessors = META_KNOWLEDGE["preprocessor"].unique().tolist()
  classifiers = META_KNOWLEDGE["classifier"].unique().tolist()
  return compute_random_search(preprocessors, classifiers, dataset, 0, seed)


def run_rs_automl_experiment():
  r = []
  for seed in SEEDS:
    aux = pd.concat([rs_automl(d, seed) for d in D_TEST_META["dataset"].unique()])
    aux["seed"] = seed
    r.append(aux)
  r = pd.concat(r)
  r.to_csv(EXP_PATH + f"rs_automl.csv", index=False)


def oracle_meta_automl(dataset, seed):
  preprocessors, classifiers, mtf_time = oracle_meta(dataset)
  return compute_random_search(preprocessors, classifiers, dataset, mtf_time, seed)

def run_oracle_meta_automl_experiment():
  r = []
  for seed in SEEDS:
    aux = pd.concat([oracle_meta_automl(d, seed) for d in D_TEST_META["dataset"].unique()])
    aux["seed"] = seed
    r.append(aux)
  r = pd.concat(r)
  r.to_csv(EXP_PATH + f"oracle_meta_automl.csv", index=False)


def random_meta_automl(dataset, seed):
  preprocessors, classifiers, mtf_time = random_meta(dataset, seed)
  return compute_random_search(preprocessors, classifiers, dataset, mtf_time, seed)

def run_random_meta_automl_experiment():
  r = []
  for seed in SEEDS:
    aux = pd.concat([random_meta_automl(d, seed) for d in D_TEST_META["dataset"].unique()])
    aux["seed"] = seed
    r.append(aux)
  r = pd.concat(r)
  r.to_csv(EXP_PATH + f"random_meta_automl.csv", index=False)


def autosklearn_2_meta_automl(dataset, seed):
  preprocessors, classifiers, mtf_time = autosklearn_2_meta()
  return compute_random_search(preprocessors, classifiers, dataset, mtf_time, seed)


def run_autosklearn_2_meta_automl_experiment():
  r = []
  for seed in SEEDS:
    aux = pd.concat([autosklearn_2_meta_automl(d, seed) for d in D_TEST_META["dataset"].unique()])
    aux["seed"] = seed
    r.append(aux)
  r = pd.concat(r)
  r.to_csv(EXP_PATH + f"autosklearn_2_meta_automl.csv", index=False)


def landmarking_meta_automl(dataset, seed):
  preprocessors, classifiers, mtf_time = landmarking_meta(dataset)
  return compute_random_search(preprocessors, classifiers, dataset, mtf_time, seed)


def run_landmarking_meta_meta_automl_experiment():
  r = []
  for seed in SEEDS:
    aux = pd.concat([landmarking_meta_automl(d, seed) for d in D_TEST_META["dataset"].unique()])
    aux["seed"] = seed
    r.append(aux)
  r = pd.concat(r)
  r.to_csv(EXP_PATH + f"landmarking_meta_automl.csv", index=False)

In [ ]:
run_oracle_meta_automl_experiment()

In [ ]:
run_random_meta_automl_experiment()

In [ ]:
run_autosklearn_2_meta_automl_experiment()

In [ ]:
run_landmarking_meta_meta_automl_experiment()

In [ ]:
run_meta_automl_experiment([0.99, 0.95, 0.90, 0.85, 0.80, 0.75, 0.50])

In [ ]:
run_rs_automl_experiment()